# Statistical comparison of errors

In [1]:
import os
import warnings

import geopandas as gpd
import matplotlib.pyplot as plt
import momepy as mm
import numpy as np
import pandas as pd
import scipy.stats

from core import stats, utils

Create a folder for evaluation data.

In [2]:
os.makedirs("../../evaluation-buildings/", exist_ok=True)

In [ ]:
# which variables to evaluate
eval_vars = [
    "edge_count",
    "edge_length",
    "node_count",
    "avg_degree",
    "stroke_count",
    "stroke_length_sum",
    "stroke_length_max",
    "coordinate_count",
]

# which methods to evaluate
methods_to_evaluate = [
    "manual",
    # "cityseer",
    "original",
    # "osmnx",
    # "parenx-voronoi",
    # "parenx-skeletonize",
    "neatnet",
    "neatnet-buildings",
]

methods_to_compare = [
    # "cityseer",
    # "osmnx",
    # "parenx-voronoi",
    # "parenx-skeletonize",
    "neatnet",
    "neatnet-buildings",
]

Generate evaluation grids and fill the with evaluation variables. At the same time, compute Euclidean distance between each method and a manual baseline.

In [4]:
for fua in utils.fua_city:
    print(f"Generating for {fua}")
    # read in base data
    meta = utils.read_sample_data()
    geom = meta.loc[meta.eFUA_ID == fua, "geometry"]
    city = meta.loc[meta.eFUA_ID == fua, "eFUA_name"].values[0]

    gdf_orig = utils.read_original(fua)
    proj_crs = gdf_orig.crs

    # Make grid
    base_grid = utils.make_grid(fua, 9, proj_crs)

    # get info on cells with revised data
    deltas = gpd.read_file(f"../../revision/{fua}/deltas_updated.gpkg")

    # read results from all methods into dict
    methods = {}

    for method in methods_to_evaluate:
        print(f"   Reading in results for {method}")
        gdf = utils.read_results(fua, method, proj_crs)

        # print("     getting graph")
        gdf = gdf[~gdf.normalize().duplicated()].copy().reset_index(drop=True)
        G = mm.gdf_to_nx(gdf, length="length", integer_labels=True)

        nodes, edges = mm.nx_to_gdf(G)

        # add node degrees
        # print("     adding node degree")
        nodes = stats.add_node_degree(nodes, G)

        # add stroke IDs
        # print("     measuring coins")
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            if method == "parenx-skeletonize":
                coins = mm.COINS(
                    edges.set_geometry(edges.simplify(0.5)),
                    angle_threshold=120,
                    flow_mode=True,
                )
            else:
                coins = mm.COINS(edges, angle_threshold=120, flow_mode=True)
        edges["stroke_id"] = coins.stroke_attribute()
        stroke_gdf = coins.stroke_gdf()

        methods[method] = {}
        methods[method]["gdf"] = gdf
        methods[method]["graph"] = G
        methods[method]["nodes"] = nodes
        methods[method]["edges"] = edges

        ### grid with stats eval for this method only
        grid = base_grid.copy()

        # print("     measuring grid edge")
        # add ratio columns to grid
        grid[["edge_count", "edge_length", "coordinate_count"]] = grid.apply(
            lambda x: stats.get_edge_stats(edges, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring grid node")
        grid[["node_count", "node_degrees", "avg_degree"]] = grid.apply(
            lambda x: stats.get_node_stats(nodes, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring stroke")
        grid[["stroke_count", "stroke_length_sum", "stroke_length_max"]] = grid.apply(
            lambda x: stats.get_stroke_stats(edges, stroke_gdf, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # add info on which cells have to be verified still
        grid["to_verify_total"] = deltas["to_verify_total"]

        # save grid to dict
        methods[method]["grid"] = grid

        # save to a file
        grid.to_file(f"../../evaluation-buildings/{fua}.gpkg", layer=method)

Generating for 1133
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
   Reading in results for neatnet-buildings
Generating for 869
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
   Reading in results for neatnet-buildings
Generating for 4617
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
   Reading in results for neatnet-buildings
Generating for 809
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
   Reading in results for neatnet-buildings
Generating for 1656
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
   Reading in results for neatnet-buildings
Generating for 4881
   Reading in results for revised_manual
   Reading in results for original
   Reading in results for neatnet
  

## Statistical comparison

Compare the results with the manually simplified networks using xi correlation.

In [6]:
for fua in utils.fua_city:
    os.makedirs(f"../../plots/evaluation-buildings/{fua}", exist_ok=True)

    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(
            f"../../evaluation-buildings/{fua}.gpkg", layer=method
        )

    # plot the distance by eval variable
    fig, axs = plt.subplots(8, figsize=(6, 18))

    for i, eval_var in enumerate(eval_vars):
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        if (stat_result.pvalue > 0.01).any():
            print(fua, eval_var, "pvalue issue")
        stat_result.statistics.plot.barh(ax=axs.flat[i])
        axs.flat[i].axvline(stat_result.statistics.max(), color="coral", linestyle="--")
        axs.flat[i].set_xlabel(f"xi for {eval_var}")
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation-buildings/{fua}/xi_correlation.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Getting tables.

In [7]:
tables = {}
for fua in utils.fua_city:
    tables[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(
            f"../../evaluation-buildings/{fua}.gpkg", layer=method
        )

    for eval_var in eval_vars:
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare + ["original"]:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        tables[fua][eval_var] = stat_result

Combining the tables.

In [8]:
combined = pd.DataFrame(columns=["variable", "case", "original"] + methods_to_compare)
combined[combined.columns.drop("variable")] = combined[
    combined.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined.loc[i] = tables[fua][eval_var].statistics
        combined.loc[i, ["variable", "case"]] = [eval_var, fua]
        i += 1

/var/folders/66/3jkth_7d5gggg6pyr8yywwt40000gn/T/ipykernel_5869/2817628644.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'edge_count' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined.loc[i, ["variable", "case"]] = [eval_var, fua]


Preparing the final output

In [9]:
combined["case"] = combined["case"].map(utils.fua_city)

In [10]:
combined = combined.set_index(["variable", "case"]).sort_index(level=0)
combined

original   neatnet  neatnet-buildings
variable          case                                                 
avg_degree        Aleppo          0.682729  0.910745           0.912170
                  Auckland        0.439375  0.803478           0.827448
                  Bucaramanga     0.675186  0.911149           0.913228
                  Douala          0.578427  0.956306           0.957746
                  Liège           0.638495  0.849321           0.856620
                  Salt Lake City  0.396668  0.692184           0.696842
                  Wuhan           0.612158  0.728728           0.739966
coordinate_count  Aleppo          0.837659  0.915682           0.920385
                  Auckland        0.727216  0.782729           0.821710
                  Bucaramanga     0.826967  0.909425           0.921732
                  Douala          0.771643  0.957401           0.964686
                  Liège           0.817363  0.862290           0.870500
                  Salt Lake City  0.652174  0.715909           0.720577
                  Wuhan           0.628399  0.694902           0.699651
edge_count        Aleppo          0.635280  0.941876           0.941768
                  Auckland        0.403283  0.823507           0.849506
                  Bucaramanga     0.439914  0.923342           0.935933
                  Douala          0.464844  0.973422           0.978015
                  Liège           0.575733  0.861521           0.872113
                  Salt Lake City  0.397923  0.781406           0.791984
                  Wuhan           0.462786  0.736018           0.738391
edge_length       Aleppo          0.875160  0.937799           0.939693
                  Auckland        0.864817  0.787780           0.830877
                  Bucaramanga     0.895524  0.957251           0.964186
                  Douala          0.925766  0.964932           0.972443
                  Liège           0.828329  0.858211           0.863746
                  Salt Lake City  0.631732  0.766855           0.777521
                  Wuhan           0.647210  0.734786           0.733537
node_count        Aleppo          0.481027  0.952117           0.951053
                  Auckland        0.272266  0.830235           0.844011
                  Bucaramanga     0.282805  0.928557           0.933177
                  Douala          0.369398  0.972882           0.976410
                  Liège           0.464697  0.868464           0.879200
                  Salt Lake City  0.254606  0.732374           0.739207
                  Wuhan           0.338599  0.778799           0.784136
stroke_count      Aleppo          0.878125  0.934588           0.933517
                  Auckland        0.725617  0.794198           0.824670
                  Bucaramanga     0.892612  0.935236           0.944105
                  Douala          0.929293  0.964968           0.972037
                  Liège           0.818340  0.862387           0.868463
                  Salt Lake City  0.645879  0.756146           0.766542
                  Wuhan           0.659154  0.719307           0.723382
stroke_length_max Aleppo          0.872386  0.866201           0.878557
                  Auckland        0.819129  0.752030           0.784598
                  Bucaramanga     0.913375  0.879617           0.889376
                  Douala          0.930621  0.916189           0.927978
                  Liège           0.850570  0.818620           0.831131
                  Salt Lake City  0.778740  0.731902           0.752318
                  Wuhan           0.878748  0.736172           0.736963
stroke_length_sum Aleppo          0.841134  0.799527           0.819240
                  Auckland        0.726223  0.598867           0.609180
                  Bucaramanga     0.887058  0.840401           0.851111
                  Douala          0.873784  0.823384           0.839100
                  Liège           0.783179  0.714858           0.742534
  

In [12]:
combined.to_csv("combined-buildings.csv")

Doing the same for Euclidean distance between distributions

In [13]:
all_deltas = {}
for fua in utils.fua_city:
    all_deltas[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(
            f"../../evaluation-buildings/{fua}.gpkg", layer=method
        )

    # get euclidean distance between the distributions
    for eval_var in eval_vars:
        all_deltas[fua][eval_var] = {}
        for method in ["original"] + methods_to_compare:
            delta_comp = (
                data["revised_manual"][eval_var] - data[method][eval_var]
            ) ** 2

            delta = np.sqrt((delta_comp).sum())
            all_deltas[fua][eval_var][method] = delta

Plot the figures.

In [14]:
for fua in utils.fua_city:
    for eval_var in eval_vars:
        # plot the distance by eval variable
        fig, axs = plt.subplots(8, figsize=(6, 18))
        for i, eval_var in enumerate(eval_vars):
            s = pd.Series(all_deltas[fua][eval_var])
            s.plot.barh(ax=axs.flat[i])
            axs.flat[i].axvline(s.min(), color="coral", linestyle="--")
            axs.flat[i].set_xlabel(eval_var)
        axs[-1].set_xscale("log")
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation-buildings/{fua}/euclidean_distance.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Combine all together to get a single table.

In [15]:
combined_deltas = pd.DataFrame(
    columns=["variable", "case", "original"] + methods_to_compare
)
combined_deltas[combined_deltas.columns.drop("variable")] = combined_deltas[
    combined_deltas.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined_deltas.loc[i] = [eval_var, fua] + list(
            all_deltas[fua][eval_var].values()
        )
        i += 1

In [16]:
combined_deltas["case"] = combined_deltas["case"].map(utils.fua_city)

In [17]:
combined_deltas = combined_deltas.set_index(["variable", "case"]).sort_index(level=0)

In [18]:
combined_deltas = combined_deltas.round(2)
combined_deltas

original     neatnet  neatnet-buildings
variable          case                                                     
avg_degree        Aleppo               85.59       22.85              22.38
                  Auckland             33.70       18.40              16.43
                  Bucaramanga          65.62       15.97              16.15
                  Douala               38.65        8.19               8.81
                  Liège                83.43       31.75              30.78
                  Salt Lake City       43.51       26.45              25.97
                  Wuhan               130.35       71.62              70.02
coordinate_count  Aleppo             1382.17      356.36             334.88
                  Auckland           2454.29      634.07             468.56
                  Bucaramanga        2164.92      631.22             509.44
                  Douala             1614.90      212.21             188.20
                  Liège              2273.62      500.08             455.81
                  Salt Lake City     1647.17      469.12             457.49
                  Wuhan              2223.10      884.07             831.37
edge_count        Aleppo             1061.56      103.06              97.31
                  Auckland           1891.28       74.94              65.63
                  Bucaramanga        1900.47      107.46              70.13
                  Douala             1414.40       39.64              34.50
                  Liège              1718.62      109.40              96.64
                  Salt Lake City     1252.82       84.32              82.47
                  Wuhan              1436.56      278.81             250.48
edge_length       Aleppo            16519.57     6913.86            7062.86
                  Auckland           6512.24    10848.20            8296.57
                  Bucaramanga        7616.64     6299.12            5366.06
                  Douala             5597.17     3270.91            2890.56
                  Liège             12283.41     8162.74            7860.69
                  Salt Lake City    16890.06     9569.32            9472.46
                  Wuhan             36486.56    20878.50           20403.46
node_count        Aleppo              976.61       59.61              56.12
                  Auckland           1816.19       43.24              42.00
                  Bucaramanga        1851.55       77.30              43.93
                  Douala             1384.78       24.39              21.14
                  Liège              1651.35       64.51              56.84
                  Salt Lake City     1171.78       54.19              53.27
                  Wuhan              1278.59      148.38             133.29
stroke_count      Aleppo              104.24       46.25              45.90
                  Auckland             83.91       44.54              34.87
                  Bucaramanga          59.97       50.26              35.94
                  Douala               46.78       22.58              19.67
                  Liège                99.29       49.84              45.62
                  Salt Lake City       85.60       49.56              49.44
                  Wuhan               181.31      113.38             106.06
stroke_length_max Aleppo           274171.86   589014.38          578767.76
                  Auckland         103500.48   218436.45          219818.29
                  Bucaramanga      136894.67   242635.81          231986.18
                  Douala           224463.20   224007.23          226454.62
                  Liège            232284.04   635367.95          616142.51
                  Salt Lake City   240447.83   278424.21          270152.46
                  Wuhan           1025821.94  1486867.33         1479680.61
stroke_length_sum Aleppo           400970.27   630464.03          602266.62
                  Auckland         177731.19   481993.11          468431.09
        

In [19]:
combined_deltas.to_csv("combined_deltas-buildings.csv")